## Without Delta Pipeline, with Spark and Parquet

![stream](https://kpistoropen.blob.core.windows.net/collateral/delta/non-delta-new.png)

In [2]:
#quick clean up of demo folder:
dbutils.fs.rm("dbfs:/workshop/nodelta/", True)

Out[ 1 ]: True

#### Historical and new data is often written in very small files and very small directories (such as eventhub capture): 
+ This data is also partitioned by arrival time not event time!

![stream](https://docs.microsoft.com/en-us/azure/data-lake-store/media/data-lake-store-archive-eventhub-capture/data-lake-store-eventhub-data-sample.png)

# Step 0: Read data

In [5]:
%fs ls mnt/databricks-workshop-datasets/Contoso-retail/structured-streaming/events

path,name,size
dbfs:/mnt/databricks-workshop-datasets/Contoso-retail/structured-streaming/events/file-0.json,file-0.json,72530
dbfs:/mnt/databricks-workshop-datasets/Contoso-retail/structured-streaming/events/file-1.json,file-1.json,72961
dbfs:/mnt/databricks-workshop-datasets/Contoso-retail/structured-streaming/events/file-10.json,file-10.json,73025
dbfs:/mnt/databricks-workshop-datasets/Contoso-retail/structured-streaming/events/file-11.json,file-11.json,72999
dbfs:/mnt/databricks-workshop-datasets/Contoso-retail/structured-streaming/events/file-12.json,file-12.json,72987
dbfs:/mnt/databricks-workshop-datasets/Contoso-retail/structured-streaming/events/file-13.json,file-13.json,73006
dbfs:/mnt/databricks-workshop-datasets/Contoso-retail/structured-streaming/events/file-14.json,file-14.json,73003
dbfs:/mnt/databricks-workshop-datasets/Contoso-retail/structured-streaming/events/file-15.json,file-15.json,73007
dbfs:/mnt/databricks-workshop-datasets/Contoso-retail/structured-streaming/events/file-16.json,file-16.json,72978
dbfs:/mnt/databricks-workshop-datasets/Contoso-retail/structured-streaming/events/file-17.json,file-17.json,73008


In [6]:
%fs head "/mnt/databricks-workshop-datasets/Contoso-retail/structured-streaming/events/file-0.json"

[Truncated to first 65536 bytes]
{"time":1469501107,"action":"Open"}
{"time":1469501147,"action":"Open"}
{"time":1469501202,"action":"Open"}
{"time":1469501219,"action":"Open"}
{"time":1469501225,"action":"Open"}
{"time":1469501234,"action":"Open"}
{"time":1469501245,"action":"Open"}
{"time":1469501246,"action":"Open"}
{"time":1469501248,"action":"Open"}
{"time":1469501256,"action":"Open"}
{"time":1469501264,"action":"Open"}
{"time":1469501266,"action":"Open"}
{"time":1469501267,"action":"Open"}
{"time":1469501269,"action":"Open"}
{"time":1469501271,"action":"Open"}
{"time":1469501282,"action":"Open"}
{"time":1469501285,"action":"Open"}
{"time":1469501291,"action":"Open"}
{"time":1469501297,"action":"Open"}
{"time":1469501303,"action":"Open"}
{"time":1469501322,"action":"Open"}
{"time":1469501335,"action":"Open"}
{"time":1469501344,"action":"Open"}
{"time":1469501346,"action":"Open"}
{"time":1469501349,"action":"Open"}
{"time":1469501357,"action":"Open"}
{"time":1469501366,"action":"Open"}
{"time":1469501371,"action":"Open"}
{"time":1469501375,"action":"Open"}
{"time":1469501375,"action":"Open"}
{"time":1469501381,"action":"Open"}
{"time":1469501392,"action":"Open"}
{"time":1469501402,"action":"Open"}
{"time":1469501407,"action":"Open"}
{"time":1469501410,"action":"Open"}
{"time":1469501420,"action":"Open"}
{"time":1469501424,"action":"Open"}
{"time":1469501438,"action":"Open"}
{"time":1469501442,"action":"Close"}
{"time":1469501462,"action":"Open"}
{"time":1469501480,"action":"Open"}
{"time":1469501488,"action":"Open"}
{"time":1469501489,"action":"Open"}
{"time":1469501491,"action":"Open"}
{"time":1469501503,"action":"Open"}
{"time":1469501505,"action":"Open"}
{"time":1469501509,"action":"Open"}
{"time":1469501513,"action":"Open"}
{"time":1469501517,"action":"Open"}
{"time":1469501520,"action":"Open"}
{"time":1469501525,"action":"Open"}
{"time":1469501533,"action":"Open"}
{"time":1469501539,"action":"Open"}
{"time":1469501540,"action":"Open"}
{"time":1469501541,"action":"Open"}
{"time":1469501543,"action":"Open"}
{"time":1469501544,"action":"Open"}
{"time":1469501545,"action":"Close"}
{"time":1469501545,"action":"Open"}
{"time":1469501547,"action":"Open"}
{"time":1469501552,"action":"Open"}
{"time":1469501557,"action":"Open"}
{"time":1469501559,"action":"Open"}
{"time":1469501560,"action":"Open"}
{"time":1469501560,"action":"Open"}
{"time":1469501565,"action":"Open"}
{"time":1469501566,"action":"Open"}
{"time":1469501574,"action":"Open"}
{"time":1469501575,"action":"Open"}
{"time":1469501575,"action":"Open"}
{"time":1469501578,"action":"Open"}
{"time":1469501581,"action":"Open"}
{"time":1469501584,"action":"Open"}
{"time":1469501600,"action":"Open"}
{"time":1469501601,"action":"Open"}
{"time":1469501603,"action":"Open"}
{"time":1469501610,"action":"Open"}
{"time":1469501620,"action":"Open"}
{"time":1469501621,"action":"Open"}
{"time":1469501625,"action":"Open"}
{"time":1469501625,"action":"Close"}
{"time":1469501626,"action":"Open"}
{"time":1469501631,"action":"Open"}
{"time":1469501632,"action":"Open"}
{"time":1469501632,"action":"Open"}
{"time":1469501638,"action":"Open"}
{"time":1469501643,"action":"Open"}
{"time":1469501646,"action":"Open"}
{"time":1469501662,"action":"Open"}
{"time":1469501662,"action":"Open"}
{"time":1469501662,"action":"Open"}
{"time":1469501663,"action":"Open"}
{"time":1469501667,"action":"Open"}
{"time":1469501674,"action":"Open"}
{"time":1469501675,"action":"Open"}
{"time":1469501678,"action":"Close"}
{"time":1469501680,"action":"Open"}
{"time":1469501685,"action":"Open"}
{"time":1469501686,"action":"Open"}
{"time":1469501689,"action":"Open"}
{"time":1469501691,"action":"Open"}
{"time":1469501694,"action":"Open"}
{"time":1469501696,"action":"Close"}
{"time":1469501702,"action":"Open"}
{"time":1469501703,"action":"Open"}
{"time":1469501704,"action":"Open"}
{"time":1469501706,"action":"Open"}
{"time":1469501706,"action":"Open"}
{"time":1469501710,"action":"Open"}
{"time":1469501715,"action":"Open"}
{"

In [7]:
from pyspark.sql.functions import expr

rawData = spark.read \
  .option("inferSchema", "true") \
  .json("/mnt/databricks-workshop-datasets/Contoso-retail/structured-streaming/events") \
  .drop("time") \
  .withColumn("date", expr("cast(concat('2018-01-', cast(rand(5) * 30 as int) + 1) as date)")) \
  .withColumn("deviceId", expr("cast(rand(5) * 100 as int)"))
  # add a couple of columns for demo purposes

In [8]:
rawData.rdd.getNumPartitions()

Out[ 3 ]: 13

In [9]:
display(rawData)

action,date,deviceId
Close,2018-01-03,8
Close,2018-01-18,57
Open,2018-01-21,69
Close,2018-01-14,43
Open,2018-01-15,49
Open,2018-01-18,59
Close,2018-01-18,57
Close,2018-01-01,0
Close,2018-01-01,0
Open,2018-01-25,83


# Step 1: Write out raw data and create staging table

In [11]:
#Define path where to write to -- by default, in this workshop, we write to the workspace filestore
writeBase = "dbfs:/workshop/nodelta/"
writePath = writeBase + "iotPipeline/"

#If there are multiple users working on the same instance, please use this writeBase, adding your $USERNAME to the path, and to any subsequent write/read
#writeBase = writeBase = "dbfs:/workshop/nodelta/$USERNAME/"
#writePath = writeBase + "iotPipeline/"

#As backup, you can always write to this blob
#writeBase = "dbfs:/mnt/databricks-workshop-exercises/Contoso-retail/nodelta"
#writePath = writeBase + "iotPipeline/"

In [12]:
#make sure it uses writePath
rawData.write.format("parquet").partitionBy("date").save(writePath)

In [13]:
%fs ls dbfs:/workshop/nodelta/iotPipeline/

path,name,size
dbfs:/workshop/nodelta/iotPipeline/_SUCCESS,_SUCCESS,0
dbfs:/workshop/nodelta/iotPipeline/date=2018-01-01/,date=2018-01-01/,0
dbfs:/workshop/nodelta/iotPipeline/date=2018-01-02/,date=2018-01-02/,0
dbfs:/workshop/nodelta/iotPipeline/date=2018-01-03/,date=2018-01-03/,0
dbfs:/workshop/nodelta/iotPipeline/date=2018-01-04/,date=2018-01-04/,0
dbfs:/workshop/nodelta/iotPipeline/date=2018-01-05/,date=2018-01-05/,0
dbfs:/workshop/nodelta/iotPipeline/date=2018-01-06/,date=2018-01-06/,0
dbfs:/workshop/nodelta/iotPipeline/date=2018-01-07/,date=2018-01-07/,0
dbfs:/workshop/nodelta/iotPipeline/date=2018-01-08/,date=2018-01-08/,0
dbfs:/workshop/nodelta/iotPipeline/date=2018-01-09/,date=2018-01-09/,0


In [14]:
%fs ls dbfs:/workshop/nodelta/iotPipeline/date=2018-01-01

ls: cannot access 'dbfs:/workshop/nodelta/iotPipeline/date=2018-01-01': No such file or directory

In [15]:
%sql
-- make sure it uses writePath
DROP TABLE IF EXISTS demo_iot_data;
CREATE TABLE demo_iot_data (action STRING, date DATE, deviceID INTEGER)
USING parquet
OPTIONS (path = "dbfs:/workshop/nodelta/iotPipeline/")
PARTITIONED BY (date)

# Step 2: Query the data

In [17]:
%sql
SELECT count(*) FROM demo_iot_data

count(1)
0


Wait, no results? That's strange. Let's repair the table then.

In [19]:
%sql
SHOW PARTITIONS demo_iot_data

partition


In [20]:
%sql

MSCK REPAIR TABLE demo_iot_data

In [21]:
%sql
SHOW PARTITIONS demo_iot_data

partition
date=2018-01-01
date=2018-01-02
date=2018-01-03
date=2018-01-04
date=2018-01-05
date=2018-01-06
date=2018-01-07
date=2018-01-08
date=2018-01-09
date=2018-01-10


In [22]:
%sql

SELECT count(*) FROM demo_iot_data

count(1)
100000


# Step 3: Appending new data

In [24]:
new_data = spark.range(100000) \
  .selectExpr("'Open' as action", "'2018-01-30' date") \
  .withColumn("deviceId", expr("cast(rand(5) * 500 as int)"))

In [25]:
display(new_data)

action,date,deviceId
Open,2018-01-30,43
Open,2018-01-30,289
Open,2018-01-30,348
Open,2018-01-30,219
Open,2018-01-30,247
Open,2018-01-30,296
Open,2018-01-30,289
Open,2018-01-30,3
Open,2018-01-30,3
Open,2018-01-30,415


##Note: This is dangerous to simply append to the production table.

In [27]:
new_data.write.format("parquet").partitionBy("date").mode("append").save(writePath)

In [28]:
%fs ls dbfs:/workshop/nodelta/iotPipeline/

path,name,size
dbfs:/workshop/nodelta/iotPipeline/_SUCCESS,_SUCCESS,0
dbfs:/workshop/nodelta/iotPipeline/date=2018-01-01/,date=2018-01-01/,0
dbfs:/workshop/nodelta/iotPipeline/date=2018-01-02/,date=2018-01-02/,0
dbfs:/workshop/nodelta/iotPipeline/date=2018-01-03/,date=2018-01-03/,0
dbfs:/workshop/nodelta/iotPipeline/date=2018-01-04/,date=2018-01-04/,0
dbfs:/workshop/nodelta/iotPipeline/date=2018-01-05/,date=2018-01-05/,0
dbfs:/workshop/nodelta/iotPipeline/date=2018-01-06/,date=2018-01-06/,0
dbfs:/workshop/nodelta/iotPipeline/date=2018-01-07/,date=2018-01-07/,0
dbfs:/workshop/nodelta/iotPipeline/date=2018-01-08/,date=2018-01-08/,0
dbfs:/workshop/nodelta/iotPipeline/date=2018-01-09/,date=2018-01-09/,0


# Step 4: Query should show new results

In [30]:
%sql

SELECT count(*) FROM demo_iot_data

count(1)
100000


That's strange, well, we can repair the table again right.

In [32]:
%sql

MSCK REPAIR TABLE demo_iot_data

In [33]:
%sql

SELECT count(*) FROM demo_iot_data

count(1)
200000


# Step 5: Upserts / Changes (on previously written data)

In [35]:
new_data.count()

Out[ 11 ]: 100000

In [36]:
new_data.drop("date").write.format("parquet").mode("overwrite").save(writePath + "date=2018-01-30/")

# Step 6: Query should reflect new data

In [38]:
%sql

SELECT count(*) FROM demo_iot_data

com.databricks.backend.common.rpc.DatabricksExceptions$SQLExecutionException: org.apache.spark.SparkException: Job aborted due to stage failure: Task 2 in stage 630.0 failed 4 times, most recent failure: Lost task 2.3 in stage 630.0 (TID 5769, 10.179.0.6, executor 4): java.io.FileNotFoundException: dbfs:/workshop/nodelta/iotPipeline/date=2018-01-30/part-00005-tid-6411181166848381321-6a14ea33-b864-46a0-a5f8-84b1313ef3f5-4955.c000.snappy.parquet
It is possible the underlying files have been updated. You can explicitly invalidate the cache in Spark by running 'REFRESH TABLE tableName' command in SQL or by recreating the Dataset/DataFrame involved.
	at org.apache.spark.sql.execution.datasources.FileScanRDD$$anon$1$$anon$2.getNext(FileScanRDD.scala:240)
	at org.apache.spark.util.NextIterator.hasNext(NextIterator.scala:73)
	at org.apache.spark.sql.execution.datasources.FileScanRDD$$anon$1$$anonfun$prepareNextFile$1.apply(FileScanRDD.scala:389)
	at org.apache.spark.sql.execution.datasources.FileScanRDD$$anon$1$$anonfun$prepareNextFile$1.apply(FileScanRDD.scala:377)
	at scala.concurrent.impl.Future$PromiseCompletingRunnable.liftedTree1$1(Future.scala:24)
	at scala.concurrent.impl.Future$PromiseCompletingRunnable.run(Future.scala:24)
	at org.apache.spark.util.threads.SparkThreadLocalCapturingRunnable$$anonfun$run$1.apply$mcV$sp(SparkThreadLocalForwardingThreadPoolExecutor.scala:82)
	at org.apache.spark.util.threads.SparkThreadLocalCapturingRunnable$$anonfun$run$1.apply(SparkThreadLocalForwardingThreadPoolExecutor.scala:82)
	at org.apache.spark.util.threads.SparkThreadLocalCapturingRunnable$$anonfun$run$1.apply(SparkThreadLocalForwardingThreadPoolExecutor.scala:82)
	at org.apache.spark.util.threads.SparkThreadLocalCapturingHelper$class.runWithCaptured(SparkThreadLocalForwardingThreadPoolExecutor.scala:52)
	at org.apache.spark.util.threads.SparkThreadLocalCapturingRunnable.runWithCaptured(SparkThreadLocalForwardingThreadPoolExecutor.scala:79)
	at org.apache.spark.util.threads.SparkThreadLocalCapturingRunnable.run(SparkThreadLocalForwardingThreadPoolExecutor.scala:82)
	at java.util.concurrent.ThreadPoolExecutor.runWorker(ThreadPoolExecutor.java:1149)
	at java.util.concurrent.ThreadPoolExecutor$Worker.run(ThreadPoolExecutor.java:624)
	at java.lang.Thread.run(Thread.java:748)
Caused by: java.io.FileNotFoundException: dbfs:/workshop/nodelta/iotPipeline/date=2018-01-30/part-00005-tid-6411181166848381321-6a14ea33-b864-46a0-a5f8-84b1313ef3f5-4955.c000.snappy.parquet
	at com.databricks.backend.daemon.data.client.DatabricksFileSystemV2$$anonfun$getFileStatus$1$$anonfun$apply$15.apply(DatabricksFileSystemV2.scala:670)
	at com.databricks.backend.daemon.data.client.DatabricksFileSystemV2$$anonfun$getFileStatus$1$$anonfun$apply$15.apply(DatabricksFileSystemV2.scala:656)
	at com.databricks.s3a.S3AExeceptionUtils$.convertAWSExceptionToJavaIOException(DatabricksStreamUtils.scala:107)
	at com.databricks.backend.daemon.data.client.DatabricksFileSystemV2$$anonfun$getFileStatus$1.apply(DatabricksFileSystemV2.scala:656)
	at com.databricks.backend.daemon.data.client.DatabricksFileSystemV2$$anonfun$getFileStatus$1.apply(DatabricksFileSystemV2.scala:656)
	at com.databricks.logging.UsageLogging$$anonfun$recordOperation$1.apply(UsageLogging.scala:359)
	at com.databricks.logging.UsageLogging$$anonfun$withAttributionContext$1.apply(UsageLogging.scala:235)
	at scala.util.DynamicVariable.withValue(DynamicVariable.scala:58)
	at com.databricks.logging.UsageLogging$class.withAttributionContext(UsageLogging.scala:230)
	at com.databricks.backend.daemon.data.client.DatabricksFileSystemV2.withAttributionContext(DatabricksFileSystemV2.scala:415)
	at com.databricks.logging.UsageLogging$class.withAttributionTags(UsageLogging.scala:268)
	at com.databricks.backend.daemon.data.client.DatabricksFileSystemV2.withAttributionTags(DatabricksFileSystemV2.scala:415)
	at com.databricks.logging.UsageLogging$class.recordOperation(UsageLogging.scala:345)
	at com.databricks.backend.daemon.da

That's strange, guess we need to refresh the metadata.

In [40]:
%sql

REFRESH TABLE demo_iot_data

In [41]:
%sql

SELECT count(*) FROM demo_iot_data

# Step 7: Add historical data

In [43]:
from pyspark.sql.functions import expr
  
old_batch_data = spark.range(100000) \
  .repartition(200) \
  .selectExpr("'Open' as action", "cast(concat('2018-01-', cast(rand(5) * 15 as int) + 1) as date) as date") \
  .withColumn("deviceId", expr("cast(rand(5) * 100 as int)"))

old_batch_data.write.format("parquet").partitionBy("date").mode("append").save(writePath)

In [44]:
%sql

SELECT count(*) FROM demo_iot_data

Won't be up to date until we call refresh

In [46]:
%sql

REFRESH TABLE demo_iot_data

In [47]:
%sql

SELECT count(*) FROM demo_iot_data

# Performance Improvements

Now we want to build other pipelines with this information, we want to write out to our data lake and allow data scientists to query it quickly. The above query took 7 seconds, there's not much data there - it's probably just not well formatted.

In order to get reasonable performance you're going to have to build a whole pipeline just to manage the file sizes and trying to optimize it for querying later on.

In [49]:
%fs ls dbfs:/workshop/nodelta/iotPipeline/

In [50]:
%fs ls dbfs:/workshop/nodelta/iotPipeline/date=2018-01-01/	

### Use the cell below as the benchmark for a no-delta pipeline

In [52]:
%sql

SELECT count(*) FROM demo_iot_data

## Next Step

[Simple Pipeline with Delta]($../5-Delta/5-02 Simple Pipeline with Delta)

&copy; 2018 Databricks, Inc. All rights reserved.<br/>
Apache, Apache Spark, Spark and the Spark logo are trademarks of the <a href="http://www.apache.org/">Apache Software Foundation</a>.<br/>
<br/>
<a href="https://databricks.com/privacy-policy">Privacy Policy</a> | <a href="https://databricks.com/terms-of-use">Terms of Use</a> | <a href="http://help.databricks.com/">Support</a>